# **PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS - NÚCLEO DE EDUCAÇÃO A DISTÂNCIA**

### Pós-graduação Lato Sensu em Ciência de Dados e Big Data


# Clusterização do índice de vacinação no estado do Maranhão com base em indicadores Sócio-econômicos



### Aluno: José de Ribamar Mendes Farias

## 3. Processamento/Tratamento de Dados

Obtidos os arquivos com os dados que servirão de base para construção do modelo, apresentamos a etapa de processamento e tratamento dos dados, que detalha o processo utilizado.

## 3.1 Bibliotecas

In [1]:
import pandas as pd
from datetime import datetime, date
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 3.2 Tratamento Dataset Registros da Vacinação no Maranhão (openDataSUS)


A vacinação contra a Covid-19 no Maranhão iniciou em 18/01/2021, com a aplicação da primeira dose da Coronavac, produzida pelo Butantan em parceria com o laboratório chinês Sinovac, em cinco maranhenses: 
- quatro profissionais da saúde e.
- uma indígena.

O presente trabalho, a partir deste fato, tenta analisar o desempenho da vacinação contra Covid-19 durante o ano 2021.

A integração das bases de dados disponibilizadas pelo openDataSUS está documentada no notebook: TCC_PUC_00 - Vacina_MA - Carga dos dados da vacinação.ipynb.

Vamos ao tratamento dos registros.

## 3.2.1 Carregando a planilha com os registros da vacinação Covid-19/MA

In [2]:
# Carregando a planilha registros da vacinação Covid-19/MA (extração 19/02/2022)
df_regvac = pd.read_csv('RegVacina_MA.csv', encoding='utf-8', sep=',', header=0, low_memory=False)

## 3.2.2 Verificando os dados carregados

In [3]:
df_regvac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564664 entries, 0 to 9564663
Data columns (total 33 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Unnamed: 0                                int64  
 1   document_id                               object 
 2   paciente_id                               object 
 3   paciente_idade                            float64
 4   paciente_dataNascimento                   object 
 5   paciente_enumSexoBiologico                object 
 6   paciente_racaCor_codigo                   float64
 7   paciente_racaCor_valor                    object 
 8   paciente_endereco_coIbgeMunicipio         object 
 9   paciente_endereco_coPais                  object 
 10  paciente_endereco_nmMunicipio             object 
 11  paciente_endereco_nmPais                  object 
 12  paciente_endereco_uf                      object 
 13  paciente_endereco_cep                     object 
 14  pa

In [4]:
# Excluindo os registros de pacientes com id ausente
df_regvac.dropna(subset=['paciente_id'], inplace=True)

Removidos 03 registros sem os dados básicos mínimos do paciente.


In [5]:
# Criando cópia do DataFrame
df_regvac2 = df_regvac.copy()

O total de registros desta base de dados é de 9.564.664 linhas em 33 colunas

## 3.2.3 Definindo o escopo do projeto - vacinações 2021

In [6]:
# Modificando as colunas das datas para o formato datatime
df_regvac2['paciente_dataNascimento'] = pd.to_datetime(df_regvac2['paciente_dataNascimento'], format='%Y/%m/%d')
df_regvac2['vacina_dataAplicacao'] = pd.to_datetime(df_regvac2['vacina_dataAplicacao'], format='%Y/%m/%d')

In [7]:
# Definindo como base de estudo - vacinações em 2021
df_regvac2 =  df_regvac2.query('vacina_dataAplicacao >= "2021-01-01" & vacina_dataAplicacao <= "2021-12-31"')

pd.unique(df_regvac2['vacina_dataAplicacao'].dt.year)

array([2021], dtype=int64)

In [8]:
df_regvac2.shape

(8710112, 33)

Limitamos nosso estudo ao ano de 2021. \
Desta forma, o dataset reduziu para 8.710.112 linhas em 33 colunas

## 3.2.4 Verificando a origem dos pacientes registrados

In [9]:
df_regvac2.paciente_endereco_uf.value_counts()

MA      8033327
PI       127926
PA        96704
SP        77645
GO        59507
TO        37367
MT        32344
DF        28101
RJ        23838
MG        21679
CE        20557
SC        11805
BA         8886
PR         8885
PE         8810
PB         6868
AM         6420
AP         5565
RO         5172
RS         5146
RR         5103
RN         4748
MS         4446
AL         2412
ES         2376
SE         1742
AC          564
XX          233
None        126
Name: paciente_endereco_uf, dtype: int64

A predominância dos pacientes vacinados é maranhense (92,2%), mas temos registros de todos estados do Brasil.\
Observamos alguns registros com este atributo ausente ou assinado incorretamente.

In [10]:
# Encontrando de pacientes com UF ausente ou incorretas
key_query = 'paciente_endereco_uf.str.contains("XX", na=False)'
df_uf_err = df_regvac2.query(key_query, engine='python')[['paciente_endereco_uf','paciente_endereco_coIbgeMunicipio',
                                                          'paciente_endereco_nmMunicipio','vacina_dataAplicacao','vacina_nome']]
df_uf_err

,paciente_endereco_uf,paciente_endereco_coIbgeMunicipio,paciente_endereco_nmMunicipio,vacina_dataAplicacao,vacina_nome
10622,XX,999999,INVALIDO,2021-04-01,COVID-19 SINOVAC/BUTANTAN - CORONAVAC
50087,XX,999999,INVALIDO,2021-05-01,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD
129978,XX,999999,INVALIDO,2021-05-24,COVID-19 ASTRAZENECA - ChAdOx1-S
192395,XX,999999,INVALIDO,2021-06-13,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD
204649,XX,999999,INVALIDO,2021-10-20,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD
...,...,...,...,...,...
9499128,XX,999999,INVALIDO,2021-06-22,COVID-19 PFIZER - COMIRNATY
9526394,XX,999999,INVALIDO,2021-06-11,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD
9536546,XX,999999,INVALIDO,2021-04-09,COVID-19 SINOVAC/BUTANTAN - CORONAVAC
9554895,XX,999999,INVALIDO,2021-04-29,COVID-19 SINOVAC/BUTANTAN - CORONAVAC


A informação da UF dos pacientes registrados com XX poderia ser recuperada com os dois primeiros dígitos do atributo código do IGBE (paciente_endereco_coIbgeMunicipio) ou pelo nome do município (paciente_endereco_nmMunicipio), mas neste caso, esta opção se mostrou inviável.

Optou-se pela exclusão dos registros. 

### 3.2.4.1 Excluindo registros incorretos ou não informados

In [11]:
# Excluindo os registros de pacientes com UF ausente ou incorretas
df_regvac2 = df_regvac2.query('paciente_endereco_uf.str.contains("None") == False', engine='python')
df_regvac2 = df_regvac2.query('paciente_endereco_uf.str.contains("XX") == False', engine='python')

In [12]:
df_regvac2.paciente_endereco_uf.value_counts()

MA    8033327
PI     127926
PA      96704
SP      77645
GO      59507
TO      37367
MT      32344
DF      28101
RJ      23838
MG      21679
CE      20557
SC      11805
BA       8886
PR       8885
PE       8810
PB       6868
AM       6420
AP       5565
RO       5172
RS       5146
RR       5103
RN       4748
MS       4446
AL       2412
ES       2376
SE       1742
AC        564
Name: paciente_endereco_uf, dtype: int64

## 3.2.5 Excluindo colunas que não serão usadas no projeto

In [13]:
# Eliminando as colunas que não serão tratadas neste trabalho

col_drop = ['Unnamed: 0','document_id','paciente_racaCor_codigo','paciente_endereco_coPais','paciente_endereco_nmPais',
            'paciente_endereco_uf','paciente_endereco_cep','paciente_nacionalidade_enumNacionalidade','estabelecimento_valor',
            'estabelecimento_razaoSocial','estalecimento_noFantasia','estabelecimento_municipio_codigo','estabelecimento_uf',
            'vacina_grupoAtendimento_codigo','vacina_grupoAtendimento_nome','vacina_categoria_codigo','vacina_lote',
            'vacina_fabricante_nome','vacina_fabricante_referencia', 'vacina_codigo','sistema_origem']

df_regvac2 = df_regvac2.drop(col_drop, axis=1)

## 3.2.6 Ajustando o dataset 

### 3.2.6.1 Renomeando as colunas para facilitar 

In [14]:
# Renomeando as colunas para simplificar
df_regvac2 = df_regvac2.rename(columns={'paciente_dataNascimento':'paciente_dt_nasc',
                                        'paciente_enumSexoBiologico':'paciente_sexo',
                                        'paciente_racaCor_valor':'paciente_raca',
                                        'paciente_endereco_coIbgeMunicipio':'paciente_cod_municp',
                                        'paciente_endereco_nmMunicipio':'paciente_nm_municp',
                                        'estabelecimento_municipio_nome':'vacina_nm_municp',
                                        'vacina_categoria_nome':'vacina_categoria',
                                        'vacina_dataAplicacao':'vacina_dt_aplic',
                                        'vacina_descricao_dose':'vacina_desc_dose'})                               


In [15]:
df_regvac2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8647943 entries, 0 to 9564663
Data columns (total 12 columns):
 #   Column               Dtype         
---  ------               -----         
 0   paciente_id          object        
 1   paciente_idade       float64       
 2   paciente_dt_nasc     datetime64[ns]
 3   paciente_sexo        object        
 4   paciente_raca        object        
 5   paciente_cod_municp  object        
 6   paciente_nm_municp   object        
 7   vacina_nm_municp     object        
 8   vacina_categoria     object        
 9   vacina_dt_aplic      datetime64[ns]
 10  vacina_desc_dose     object        
 11  vacina_nome          object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 857.7+ MB


### 3.2.6.2 Substituindo o nome das vacinas

In [16]:
dic_nmvacina = {'COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD':'AstraZeneca','COVID-19 SINOVAC/BUTANTAN - CORONAVAC':'Coronavac',
                'COVID-19 PEDIÁTRICA - PFIZER COMIRNATY':'Pfizer','COVID-19 PFIZER - COMIRNATY':'Pfizer',
                'COVID-19 ASTRAZENECA - ChAdOx1-S':'AstraZeneca','COVID-19 JANSSEN - Ad26.COV2.S':'Janssen'}
df_regvac2 = df_regvac2.replace({'vacina_nome':dic_nmvacina})

Para facilitar a leitura, substituimos o nome comercial pelo nome popular da vacina.

## 3.2.7 Excluindo registros de vacinação em crianças abaixo de 12 anos¶

A vacinação pediátrica no Maranhão foi iniciada oficialmente em 20/01/2022. Como nosso estudo está focado no ano 2021, os registros da faixa etária até 11 anos serão excluídos.

In [17]:
# Definindo idade = 12 anos como idade mínima dos pacientes
idade_min = 12.0
df_regvac2 = df_regvac2.query('paciente_idade >= @idade_min')

In [18]:
df_regvac2.isnull().sum()

paciente_id               0
paciente_idade            0
paciente_dt_nasc          0
paciente_sexo             0
paciente_raca             0
paciente_cod_municp       0
paciente_nm_municp        0
vacina_nm_municp          0
vacina_categoria       5086
vacina_dt_aplic           0
vacina_desc_dose          0
vacina_nome               0
dtype: int64

## 3.2.8 Tratamento da categoria da vacina¶

In [19]:
df_regvac2.vacina_categoria.value_counts()

Faixa Etária                                       6702409
Comorbidades                                        505453
Trabalhadores de Saúde                              430448
Trabalhadores da Educação                           354993
Povos e Comunidades Tradicionais                    255699
Trabalhadores Industriais                            98787
Gestantes                                            49228
Trabalhadores de Transporte                          39886
Forças de Segurança e Salvamento                     38344
Pessoas com Deficiência                              29897
Pessoas de 60 anos ou mais institucionalizadas       27369
Povos Indígenas                                      27357
Outros                                               20508
Trabalhadores Portuários                             17675
Trabalhadores de Limpeza Urbana                      13309
Puérperas                                            11191
Funcionário do Sistema de Privação de Liberdade       90

O atributo vacina_categoria informa o grupo atendido na vacinação, o que definia as prioridades das pessoas vacinadas, mas observamos que este atributo está fortemente relacionado ao paciente.\
Resolvemos renomear o atributo.

In [20]:
df_regvac2 = df_regvac2.rename(columns={'vacina_categoria':'paciente_categoria'})

In [21]:
df_regvac2 = df_regvac2.reindex(columns=['paciente_id','paciente_idade','paciente_dt_nasc','paciente_sexo','paciente_raca',
                                         'paciente_cod_municp','paciente_nm_municp','paciente_categoria','vacina_nm_municp',
                                         'vacina_dt_aplic','vacina_desc_dose','vacina_nome'])

In [22]:
df_regvac2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8646480 entries, 0 to 9564663
Data columns (total 12 columns):
 #   Column               Dtype         
---  ------               -----         
 0   paciente_id          object        
 1   paciente_idade       float64       
 2   paciente_dt_nasc     datetime64[ns]
 3   paciente_sexo        object        
 4   paciente_raca        object        
 5   paciente_cod_municp  object        
 6   paciente_nm_municp   object        
 7   paciente_categoria   object        
 8   vacina_nm_municp     object        
 9   vacina_dt_aplic      datetime64[ns]
 10  vacina_desc_dose     object        
 11  vacina_nome          object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 857.6+ MB


Esta será a estrutura básica do nosso dataset Registro de Vacinas.

## 3.2.9 Verificando registros duplicados

In [23]:
# Verificando a quantidade de registros duplicados
df_regvac2.duplicated().sum()

7613

In [24]:
df_regvac2.reset_index(drop=True, inplace=True)

In [25]:
ck_duplic = ['paciente_id','paciente_idade','paciente_dt_nasc','paciente_sexo','paciente_raca','vacina_nm_municp',
             'vacina_dt_aplic','vacina_desc_dose']
df_regvac2[df_regvac2.duplicated(subset=ck_duplic)]

,paciente_id,paciente_idade,paciente_dt_nasc,paciente_sexo,paciente_raca,paciente_cod_municp,paciente_nm_municp,paciente_categoria,vacina_nm_municp,vacina_dt_aplic,vacina_desc_dose,vacina_nome
85324,c91177b9d304b2261724a068504465dc08cfdcb41294e1...,46.0,1975-02-22,F,AMARELA,210930,PRESIDENTE VARGAS,Trabalhadores de Saúde,PRESIDENTE VARGAS,2021-06-10,1ª Dose,AstraZeneca
86641,1d8916336c2e2a869a9549b422d710b2e18d16db935b66...,50.0,1971-04-14,F,AMARELA,210232,BURITICUPU,Faixa Etária,BURITICUPU,2021-10-14,1ª Dose,Pfizer
105854,ffc7246ae37d4e61f286b18a987917a333805e8b90139f...,15.0,2006-01-13,F,PARDA,210232,BURITICUPU,Faixa Etária,SANTA INES,2021-11-22,2ª Dose,Pfizer
145351,684ac5e1e170758fe303fa37000fba0a32fa82ba437786...,91.0,1929-11-29,F,PARDA,211270,VARGEM GRANDE,Faixa Etária,VARGEM GRANDE,2021-04-12,2ª Dose,Coronavac
175801,ff47c308864cd3ece5b44dbf3591e5132dea3f49981f31...,76.0,1945-02-28,F,AMARELA,510263,CAMPO NOVO DO PARECIS,Faixa Etária,TIMON,2021-09-28,1ª Dose,Coronavac
...,...,...,...,...,...,...,...,...,...,...,...,...
8645889,02c02ef7a4132047acb11361f0b24d8843a3b0dc12a208...,37.0,1983-05-08,F,PARDA,211130,SAO LUIS,Trabalhadores da Educação,PRESIDENTE VARGAS,2021-05-05,1ª Dose,AstraZeneca
8646158,5219a80c4d4f108d677260f27f88c5aac4d8be05cedd90...,69.0,1951-05-07,F,PARDA,210047,ALTO ALEGRE DO PINDARE,NaN,ALTO ALEGRE DO PINDARE,2021-03-23,1ª Dose,Coronavac
8646176,3019acd7e18294827e03b532eef4db2fc70b297918b292...,42.0,1978-11-17,F,AMARELA,210930,PRESIDENTE VARGAS,Faixa Etária,PRESIDENTE VARGAS,2021-07-15,Dose,Janssen
8646294,a76ef4e01f0088821cd84673e2b6905d6ffde1c9dfcca4...,81.0,1940-01-20,M,BRANCA,210920,PRESIDENTE JUSCELINO,Faixa Etária,PRESIDENTE JUSCELINO,2021-03-09,1ª Dose,Coronavac


In [26]:
df_regvac2.loc[:,['paciente_id','paciente_dt_nasc','paciente_nm_municp','vacina_dt_aplic','vacina_desc_dose']]

,paciente_id,paciente_dt_nasc,paciente_nm_municp,vacina_dt_aplic,vacina_desc_dose
0,57ef873fe42e8f297cdb99c28d2f0815756b41d4ee632b...,1976-08-02,MIRANDA DO NORTE,2021-12-22,1ª Dose
1,b54c174fb4491eed8b83c67e6daf153b76e48c4c2c31f2...,1969-07-24,SAO LUIS,2021-08-25,2ª Dose
2,82f6fda10ff79424c50a75a25389fefd199db1527d7f20...,1969-03-22,SAO LUIS,2021-08-24,2ª Dose
3,9bf4a3972748b5f60654bc2920206985a53ad5108c6e49...,1950-07-30,PERITORO,2021-10-27,Reforço
4,312943d0aadee9b6c00dc18ed0f45aef884047188baf2e...,1962-11-13,SAO LUIS,2021-04-23,1ª Dose
...,...,...,...,...,...
8646475,f19e213a3969ccf819fb9a96c597673f8f0d01be9108a9...,1945-11-15,LAGO DA PEDRA,2021-03-29,1ª Dose
8646476,cc95279445065d3d4bb1ee9451c0f9cd8e327ddec8cd38...,1977-11-28,SAO LUIS,2021-04-25,1ª Dose
8646477,54ad29220638a1802b334251cd9fef0941f696c8dbe1e8...,2005-12-02,ROSARIO,2021-07-23,1ª Dose
8646478,cae814e1c00c6abb1787ae1604acee8734acd8b1068108...,2001-01-30,SAO LUIS,2021-08-13,1ª Dose


In [27]:
df_regvac2.sort_values('paciente_id', inplace=True)

In [28]:
df_regvac2.paciente_id.value_counts()

357fb31c6bda2296e11d6c2545bec1dd2e443a924985ccd954ae73475c6c45fb    8
f30dce82db0c8fc285117e50531f40b6797ac8362e653cb692fcf459fb1b9b62    8
4d7f46a384ef4a4b0b2ae21bc51c8eadcf81add910f73d4a51cad51b48403690    8
fc663f20fadd69fcf11d3231e4abc53753ef8627cc2824347eb701d763adf6ac    7
c7a906583c49472af301351bb0b598288f1ab9180d2f35e3139bc2b9a4cf8835    7
                                                                   ..
3e9ce07c52718a472e9e85d94afe9e985f84fed4f4fd0fbab87be376b51a9c14    1
038f8d6cd57b526b1b77f1094097a88708146f88354fa97109ed939446cadeee    1
e0e942e908e3cc2d69ce73eda5b0aa993f86dbdab8fb053cdff6afe2ac2057d5    1
4aff1cc25fb04e4f61cd1396a28383b2514698a8b9d8def8847ad1ff3c2fa4a2    1
256699429b3bb51a21bf52efd43abf31b1fac1b921a9691fe37479c734df6b33    1
Name: paciente_id, Length: 4614250, dtype: int64

Normalmente um paciente deveria ter 3 registros de vacinação, correspondentes a 1ª e 2ª doses e dose reforço.

Conforme acima, observamos que a quantidade total de pacientes é 4.614.250, entretanto alguns pacientes possuem registros acima do esperado.

Para tentar resolver esta situação resolvemos criar novas bases, por doses (1º dose, 2ª dose e dose reforço). 

## 3.2.10 Separando os registros por doses

### 3.2.10.1 DataFrame 1ª dose

In [29]:
# Criando DataFrame para receber informações da 1ª dose
df_regvac_D1 = df_regvac2.query('vacina_desc_dose == "1ª Dose" ')

In [30]:
df_regvac_D1.vacina_desc_dose.value_counts()

1ª Dose    4432440
Name: vacina_desc_dose, dtype: int64

In [31]:
df_regvac_D1.head()

,paciente_id,paciente_idade,paciente_dt_nasc,paciente_sexo,paciente_raca,paciente_cod_municp,paciente_nm_municp,paciente_categoria,vacina_nm_municp,vacina_dt_aplic,vacina_desc_dose,vacina_nome
6274449,000004050e20191103d19e89b76542ac762c35912838ad...,14.0,2007-09-28,M,AMARELA,210340,COELHO NETO,Faixa Etária,COELHO NETO,2021-10-08,1ª Dose,Pfizer
3277453,0000073e1d449a8f1ae24b5ceac7f4ac5bfd1daf2203ce...,57.0,1963-11-03,M,BRANCA,211130,SAO LUIS,Faixa Etária,SAO LUIS,2021-05-27,1ª Dose,Pfizer
5751671,00000c08e215dfabf66a513ada8cce634347971b469333...,16.0,2005-08-29,M,AMARELA,211130,SAO LUIS,Faixa Etária,ANAJATUBA,2021-12-21,1ª Dose,Pfizer
8071057,00000daced6a84b616569aeab4bda96f74971c8ebd31b6...,29.0,1992-03-06,M,AMARELA,210340,COELHO NETO,Faixa Etária,COELHO NETO,2021-11-11,1ª Dose,Coronavac
931857,00000ee6399e2172e8eaa164c22f5e6a6e86cbdab71a76...,39.0,1982-02-14,F,AMARELA,211130,SAO LUIS,Faixa Etária,SAO LUIS,2021-07-01,1ª Dose,Pfizer


In [32]:
# Renomeando as colunas de atributos das vacinas
df_regvac_D1 = df_regvac_D1.rename(columns={'vacina_nm_municp':'vacina_nm_municp_d1',
                                            'vacina_dt_aplic':'vacina_dt_aplic_d1',
                                            'vacina_desc_dose':'vacina_desc_dose_d1',
                                            'vacina_nome':'vacina_nome_d1'})                               


In [33]:
df_regvac_D1['vacina_desc_dose_d1'].value_counts()

1ª Dose    4432440
Name: vacina_desc_dose_d1, dtype: int64

In [34]:
df_regvac_D1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4432440 entries, 6274449 to 4453538
Data columns (total 12 columns):
 #   Column               Dtype         
---  ------               -----         
 0   paciente_id          object        
 1   paciente_idade       float64       
 2   paciente_dt_nasc     datetime64[ns]
 3   paciente_sexo        object        
 4   paciente_raca        object        
 5   paciente_cod_municp  object        
 6   paciente_nm_municp   object        
 7   paciente_categoria   object        
 8   vacina_nm_municp_d1  object        
 9   vacina_dt_aplic_d1   datetime64[ns]
 10  vacina_desc_dose_d1  object        
 11  vacina_nome_d1       object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 439.6+ MB


### 3.2.10.2 DataFrame 2ª dose

In [35]:
# Criando DataFrames para receber informações da 2ª dose, dose Janssen e adicionais
df_regvac_D2 = df_regvac2.query('vacina_desc_dose == "2ª Dose" ')
df_regvac_D2d = df_regvac2.query('vacina_desc_dose == "Dose" ')                                 
df_regvac_D2ad = df_regvac2.query('vacina_desc_dose == "Dose Adicional" ')                                 
df_regvac_D21d = df_regvac2.query('vacina_desc_dose == "1ª Dose Revacinação" ')

In [36]:
# Concatenando as bases com as informações da 2ª dose, dose Janssen e adicionais
df_regvac_D2 = pd.concat([df_regvac_D2,df_regvac_D2d,df_regvac_D2ad,df_regvac_D21d],ignore_index=True)

In [37]:
df_regvac_D2.vacina_desc_dose.value_counts()

2ª Dose                3539385
Dose                    110987
Dose Adicional           39619
1ª Dose Revacinação          2
Name: vacina_desc_dose, dtype: int64

In [38]:
df_regvac_D2.vacina_desc_dose.value_counts().sum()

3689993

In [39]:
# Definindo como 2ª dose o tributo vacina_desc_dose do dataset
df_regvac_D2['vacina_desc_dose'] = df_regvac_D2['vacina_desc_dose'].replace(['Dose','Dose Adicional','1ª Dose Revacinação'],
                                                                            '2ª Dose')

In [40]:
# Renomeando as colunas para simplificar
df_regvac_D2 = df_regvac_D2.rename(columns={'vacina_nm_municp':'vacina_nm_municp_d2',
                                            'vacina_dt_aplic':'vacina_dt_aplic_d2',
                                            'vacina_desc_dose':'vacina_desc_dose_d2',
                                            'vacina_nome':'vacina_nome_d2'})                               


In [41]:
df_regvac_D2['vacina_desc_dose_d2'].unique()

array(['2ª Dose'], dtype=object)

In [42]:
df_regvac_D2['vacina_desc_dose_d2'].value_counts()

2ª Dose    3689993
Name: vacina_desc_dose_d2, dtype: int64

In [43]:
df_regvac_D2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3689993 entries, 0 to 3689992
Data columns (total 12 columns):
 #   Column               Dtype         
---  ------               -----         
 0   paciente_id          object        
 1   paciente_idade       float64       
 2   paciente_dt_nasc     datetime64[ns]
 3   paciente_sexo        object        
 4   paciente_raca        object        
 5   paciente_cod_municp  object        
 6   paciente_nm_municp   object        
 7   paciente_categoria   object        
 8   vacina_nm_municp_d2  object        
 9   vacina_dt_aplic_d2   datetime64[ns]
 10  vacina_desc_dose_d2  object        
 11  vacina_nome_d2       object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 337.8+ MB


### 3.2.10.3 DataFrame dose reforço

In [44]:
# Criando Dataframes para receber informações da dose de reforço
df_regvac_Dr = df_regvac2.query('vacina_desc_dose == "Reforço" ')

In [45]:
# Renomeando as colunas para simplificar
df_regvac_Dr = df_regvac_Dr.rename(columns={'vacina_nm_municp':'vacina_nm_municp_dr',
                                            'vacina_dt_aplic':'vacina_dt_aplic_dr',
                                            'vacina_desc_dose':'vacina_desc_dose_dr',
                                            'vacina_nome':'vacina_nome_dr'})               

In [46]:
df_regvac_Dr['vacina_desc_dose_dr'].value_counts()

Reforço    524045
Name: vacina_desc_dose_dr, dtype: int64

In [47]:
df_regvac_Dr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524045 entries, 7134543 to 2290631
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   paciente_id          524045 non-null  object        
 1   paciente_idade       524045 non-null  float64       
 2   paciente_dt_nasc     524045 non-null  datetime64[ns]
 3   paciente_sexo        524045 non-null  object        
 4   paciente_raca        524045 non-null  object        
 5   paciente_cod_municp  524045 non-null  object        
 6   paciente_nm_municp   524045 non-null  object        
 7   paciente_categoria   524008 non-null  object        
 8   vacina_nm_municp_dr  524045 non-null  object        
 9   vacina_dt_aplic_dr   524045 non-null  datetime64[ns]
 10  vacina_desc_dose_dr  524045 non-null  object        
 11  vacina_nome_dr       524045 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 52.0+ M

In [48]:
df_regvac_Dr.isnull().sum()

paciente_id             0
paciente_idade          0
paciente_dt_nasc        0
paciente_sexo           0
paciente_raca           0
paciente_cod_municp     0
paciente_nm_municp      0
paciente_categoria     37
vacina_nm_municp_dr     0
vacina_dt_aplic_dr      0
vacina_desc_dose_dr     0
vacina_nome_dr          0
dtype: int64

## 3.2.11 Remontando o dataset dos registros de vacinação 

In [49]:
# Juntando os DataFrames da 1ª dose e 2ª dose
df_regvac3 = df_regvac_D1.merge(df_regvac_D2, on='paciente_id', how='outer')

Optou-se pelo full outer join para agrupar todos registros

In [50]:
df_regvac3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4681309 entries, 0 to 4681308
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   paciente_id            object        
 1   paciente_idade_x       float64       
 2   paciente_dt_nasc_x     datetime64[ns]
 3   paciente_sexo_x        object        
 4   paciente_raca_x        object        
 5   paciente_cod_municp_x  object        
 6   paciente_nm_municp_x   object        
 7   paciente_categoria_x   object        
 8   vacina_nm_municp_d1    object        
 9   vacina_dt_aplic_d1     datetime64[ns]
 10  vacina_desc_dose_d1    object        
 11  vacina_nome_d1         object        
 12  paciente_idade_y       float64       
 13  paciente_dt_nasc_y     datetime64[ns]
 14  paciente_sexo_y        object        
 15  paciente_raca_y        object        
 16  paciente_cod_municp_y  object        
 17  paciente_nm_municp_y   object        
 18  paciente_categoria_y  

In [51]:
# Criando variáveis para a cobertura vacinal
totd1 = df_regvac3.vacina_nm_municp_d1.value_counts()
totd2 = df_regvac3.vacina_nm_municp_d2.value_counts()
total = totd1 + totd2 - totd2

In [52]:
df_regvac3.vacina_desc_dose_d1.value_counts()

1ª Dose    4484090
Name: vacina_desc_dose_d1, dtype: int64

In [53]:
df_regvac3.vacina_desc_dose_d2.value_counts()

2ª Dose    3707417
Name: vacina_desc_dose_d2, dtype: int64

In [54]:
df_regvac3.vacina_desc_dose_d2.isnull().sum()

973892

##### 973.892 pacientes receberam a 1ª dose mas não retornaram para receber a 2ª dose.

In [55]:
df_regvac3 = df_regvac3.query('vacina_desc_dose_d2 != "2ª Dose" ')

In [56]:
df_regvac3.loc[:,['paciente_id','vacina_desc_dose_d1','vacina_desc_dose_d2']].head(15)

,paciente_id,vacina_desc_dose_d1,vacina_desc_dose_d2
2,00000c08e215dfabf66a513ada8cce634347971b469333...,1ª Dose,NaN
5,000012af7d1821b858c18ea6fa5979452a03a075f2b1de...,1ª Dose,NaN
20,000056c394336799c421d63bec4b68fcbc430cd1f28e60...,1ª Dose,NaN
21,0000572bf104caf9bf034d66d41d71166c53028d8a134b...,1ª Dose,NaN
24,00006066823922011f7321afe64af68df26345fc16c37a...,1ª Dose,NaN
25,000066893a0ec7cf8adbccff86b02148fc7673544e1636...,1ª Dose,NaN
28,000070d671037aa94bd574e5d2fa84fee90bc052dc10b8...,1ª Dose,NaN
31,000079a8371afb83a0e07210754da7fc03a9d92f5f2e22...,1ª Dose,NaN
33,000085618a8e185cafc576fb84cf909399ae6d5e8b320e...,1ª Dose,NaN
39,00009496d93279fdaecc1cf2bd392e1860988058f677f8...,1ª Dose,NaN


In [57]:
# Eliminando as colunas duplicadas
col_drop = ['paciente_idade_y','paciente_dt_nasc_y','paciente_sexo_y','paciente_raca_y','paciente_cod_municp_y',
            'paciente_nm_municp_y','paciente_categoria_y','vacina_nm_municp_d2','vacina_dt_aplic_d2','vacina_desc_dose_d2',
            'vacina_nome_d2']
df_regvac3 = df_regvac3.drop(col_drop, axis=1)

In [58]:
# Renomeando para o original
df_regvac3 = df_regvac3.rename(columns={'paciente_idade_x':'paciente_idade',
                                        'paciente_dt_nasc_x':'paciente_dt_nasc',
                                        'paciente_sexo_x':'paciente_sexo',
                                        'paciente_raca_x':'paciente_raca',
                                        'paciente_cod_municp_x':'paciente_cod_municp',
                                        'paciente_nm_municp_x':'paciente_nm_municp',
                                        'paciente_categoria_x':'paciente_categoria',
                                        'vacina_nm_municp_d1':'vacina_nm_municp',
                                        'vacina_dt_aplic_d1':'vacina_dt_aplic',
                                        'vacina_desc_dose_d1':'vacina_desc_dose',
                                        'vacina_nome_d1':'vacina_nome'})               

In [59]:
# Verificando registros repetidos
ck_duplic = ['paciente_id','paciente_idade','paciente_dt_nasc','paciente_sexo','paciente_raca','paciente_cod_municp',
             'paciente_nm_municp','paciente_categoria']
df_regvac3[df_regvac3.duplicated(subset=ck_duplic)]

,paciente_id,paciente_idade,paciente_dt_nasc,paciente_sexo,paciente_raca,paciente_cod_municp,paciente_nm_municp,paciente_categoria,vacina_nm_municp,vacina_dt_aplic,vacina_desc_dose,vacina_nome
3194,002f909b57c9f25259778e696644557d80b141212832ed...,57.0,1964-03-16,M,SEM INFORMACAO,210820,PEDREIRAS,Comorbidades,PEDREIRAS,2021-06-24,1ª Dose,Pfizer
3232,0030455897960f0a1a844d0b2c5237ad1a6987e6902a93...,40.0,1980-11-08,M,AMARELA,211170,SAO VICENTE FERRER,Povos e Comunidades Tradicionais,SAO VICENTE FERRER,2021-04-15,1ª Dose,AstraZeneca
4012,003b5fe2e9f980dd52c447aaaba7fe31f13354d5fc8c85...,77.0,1943-06-11,F,AMARELA,210300,CAXIAS,Faixa Etária,CAXIAS,2021-03-11,1ª Dose,AstraZeneca
4275,003f459622e160cc57cd717cb54b3381ff0448141c6200...,39.0,1981-10-26,F,PRETA,210177,BELA VISTA DO MARANHAO,Faixa Etária,BELA VISTA DO MARANHAO,2021-07-27,1ª Dose,AstraZeneca
5374,004f10b23f2a38172fbbb2fa1ff638d0e0cfc91bf24e2f...,43.0,1978-05-16,F,SEM INFORMACAO,210080,ANAPURUS,Faixa Etária,ANAPURUS,2021-07-28,1ª Dose,AstraZeneca
...,...,...,...,...,...,...,...,...,...,...,...,...
4480958,ffd391ab633c80ae6761adb34537bd641d655d4702d4ee...,73.0,1948-02-04,M,AMARELA,210405,ESTREITO,Faixa Etária,ESTREITO,2021-10-06,1ª Dose,Coronavac
4482727,ffec48763655ae52fbf7f4f8f9796acacd19a7bbcf72c1...,23.0,1998-05-04,M,AMARELA,211130,SAO LUIS,Faixa Etária,SAO LUIS,2021-08-18,1ª Dose,Coronavac
4482728,ffec48763655ae52fbf7f4f8f9796acacd19a7bbcf72c1...,23.0,1998-05-04,M,AMARELA,211130,SAO LUIS,Faixa Etária,SAO LUIS,2021-08-18,1ª Dose,Coronavac
4483578,fff8d35ce3bf049e7c74302fa8e65ae88e8718463b9be1...,49.0,1972-06-03,M,AMARELA,210310,CEDRAL,Faixa Etária,CEDRAL,2021-08-20,1ª Dose,AstraZeneca


Após a reconstrução do Dataset verificamos a existência de registros duplicados novamente. \
Encontrados 4.520 registros duplicados. \
Optou-se pela exclusão dos registros.

In [60]:
# Excluindo registros repetidos
df_regvac3 = df_regvac3.drop_duplicates(subset=ck_duplic, keep='first', inplace=False)

In [61]:
# Verificando registros repetidos
ck_duplic = ['paciente_id','paciente_idade','paciente_dt_nasc','paciente_sexo','paciente_raca','paciente_cod_municp',
             'paciente_nm_municp','paciente_categoria']
df_regvac3[df_regvac3.duplicated(subset=ck_duplic)]

,paciente_id,paciente_idade,paciente_dt_nasc,paciente_sexo,paciente_raca,paciente_cod_municp,paciente_nm_municp,paciente_categoria,vacina_nm_municp,vacina_dt_aplic,vacina_desc_dose,vacina_nome


In [62]:
df_regvac3.duplicated().sum()

0

In [63]:
df_regvac3.isnull().sum()

paciente_id              0
paciente_idade           0
paciente_dt_nasc         0
paciente_sexo            0
paciente_raca            0
paciente_cod_municp      0
paciente_nm_municp       0
paciente_categoria     357
vacina_nm_municp         0
vacina_dt_aplic          0
vacina_desc_dose         0
vacina_nome              0
dtype: int64

In [64]:
df_regvac3.reset_index(drop=True, inplace=True)

In [65]:
df_regvac3bk = df_regvac3.copy()

In [66]:
#df_regvac3 = df_regvac3bk

Considerando que nosso escopo de estudo é 2021 e o intervalo recomendado entre as doses, precisamos retirar deste grupo pacientes:
- vacinados após 01/11/2021 no caso da Pfizer ou AstraZeneca.
- vacinados após 01/12/2021 que tomaram Coronavac

In [67]:
df_regvac3a = df_regvac3.query('vacina_dt_aplic < "2021-12-01" and vacina_nome == "Coronavac" ')
df_regvac3b = df_regvac3.query('vacina_dt_aplic < "2021-11-01" and (vacina_nome == "Pfizer" or vacina_nome == "AstraZeneca")')

In [68]:
df_regvac3 = pd.concat([df_regvac3a,df_regvac3b],ignore_index=True)

In [69]:
df_regvac3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822693 entries, 0 to 822692
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   paciente_id          822693 non-null  object        
 1   paciente_idade       822693 non-null  float64       
 2   paciente_dt_nasc     822693 non-null  datetime64[ns]
 3   paciente_sexo        822693 non-null  object        
 4   paciente_raca        822693 non-null  object        
 5   paciente_cod_municp  822693 non-null  object        
 6   paciente_nm_municp   822693 non-null  object        
 7   paciente_categoria   822371 non-null  object        
 8   vacina_nm_municp     822693 non-null  object        
 9   vacina_dt_aplic      822693 non-null  datetime64[ns]
 10  vacina_desc_dose     822693 non-null  object        
 11  vacina_nome          822693 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 75.3+ MB


## 3.3 Tratamento dataset IDH Municípios (AtlasBR)

Dataset com informações socioeconômicas dos municípios brasileiros.

fonte: http://www.atlasbrasil.org.br/acervo/biblioteca

Download em 10/02/2022

In [70]:
# Importando planilha do PIB Municípios
df_idhm = pd.read_excel('Atlas 2013_municipal, estadual e Brasil.xlsx', sheet_name=1 )

In [71]:
df_idhm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16695 entries, 0 to 16694
Columns: 237 entries, ANO to IDHM_R
dtypes: float64(167), int64(69), object(1)
memory usage: 30.2+ MB


In [72]:
df_idhm.ANO.unique()

array([1991, 2000, 2010], dtype=int64)

 ### 3.3.1 Selecionando o ano 2010 e o cod UF do Maranhão

In [73]:
df_idhm2 = df_idhm.query('ANO == 2010')
df_idhm2 = df_idhm2.reset_index(drop=True)

df_idhm2 = df_idhm2.query('UF == 21')
df_idhm2 = df_idhm2.reset_index(drop=True)

In [74]:
print(df_idhm2.columns)

Index(['ANO', 'UF', 'Codmun6', 'Codmun7', 'Município', 'ESPVIDA', 'FECTOT',
       'MORT1', 'MORT5', 'RAZDEP',
       ...
       'PIA1517', 'PIA18M', 'POP', 'POPT', 'I_ESCOLARIDADE', 'I_FREQ_PROP',
       'IDHM', 'IDHM_E', 'IDHM_L', 'IDHM_R'],
      dtype='object', length=237)


### 3.3.2 Excluindo as colunas que não serão usadas

In [75]:
# Eliminando colunas que não serão usadas neste dataset 

df_idhm2.drop(columns=['ANO','UF','FECTOT','MORT5','RAZDEP','SOBRE40','SOBRE60','T_ANALF11A14','T_ANALF15A17',
                       'T_ANALF15M','T_ANALF18A24','T_ANALF25A29','T_ANALF25M','T_ENV','T_ATRASO_1_BASICO','T_ATRASO_1_FUND',
                       'T_ATRASO_1_MED','T_ATRASO_2_BASICO','T_ATRASO_2_FUND','T_ATRASO_2_MED','T_FREQ0A3','T_FREQ11A14',
                       'T_FREQ15A17','T_FREQ18A24','T_FREQ25A29','T_FREQ4A5','T_FREQ4A6','T_FREQ5A6','T_FREQ6','T_FREQ6A14',
                       'T_FREQ6A17','T_FREQFUND1517','T_FREQFUND1824','T_FREQFUND45','T_FREQMED1824','T_FREQMED614',
                       'T_FREQSUPER1517','T_FUND11A13','T_FUND12A14','T_FUND15A17','T_FUND16A18','T_FUND18A24','CORTE1',
                       'CORTE2','CORTE3','CORTE4','CORTE9','GINI','PIND','PINDCRI','PMPOB','PMPOBCRI','PPOB','PPOBCRI',
                       'PREN10RICOS','PREN20','PREN20RICOS','PREN40','PREN60','PREN80','PRENTRAB','R1040', 'R2040','RDPC1',
                       'RDPC10','RDPC2','RDPC3','RDPC4','RDPC5','RDPCT','RIND','RMPOB','RPOB','THEIL','CPR','EMP',
                       'THEILtrab','TRABCC','TRABSC','T_CRIFUNDIN_TODOS','T_FORA4A5','T_FORA6A14','T_FUNDIN_TODOS',
                       'T_FUNDIN_TODOS_MMEIO','T_FUNDIN18MINF','T_M10A14CF','T_M15A17CF','T_MULCHEFEFIF014',
                       'T_NESTUDA_NTRAB_MMEIO','T_OCUPDESLOC_1','T_RMAXIDOSO','HOMEM0A4','HOMEM10A14','HOMEM15A19',
                       'HOMEM20A24','HOMEM25A29','HOMEM30A34','HOMEM35A39','HOMEM40A44','HOMEM45A49','HOMEM50A54',
                       'HOMEM55A59','HOMEM5A9','HOMEM60A64','HOMEM65A69','HOMEM70A74','HOMEM75A79','HOMEMTOT','HOMENS80',
                       'MULH0A4','MULH10A14','MULH15A19','MULH20A24','MULH25A29','MULH30A34','MULH35A39','MULH40A44',
                       'MULH45A49','MULH50A54','MULH55A59','MULH5A9','MULH60A64','MULH65A69','MULH70A74','MULH75A79',
                       'MULHER80','MULHERTOT','PEA','PEA1014','PEA1517','peso1','PESO1114','PESO1113','PESO1214',
                       'peso13','PESO15','peso1517','PESO1524','PESO1618','Peso1820','PESO1824','Peso1921','PESO25','peso4',
                       'peso5','peso6','PESO610','Peso617','PESO65','PESOM1014','PESOM1517','PESOM15M','PESOM25M','PIA',
                       'PIA1014','PIA1517','T_FUND25M','T_MED18A20','T_MED18A24','T_MED18M','T_MED19A21','T_MED25M',
                       'T_SUPER25M','T_ATIV1014','T_ATIV1517','T_ATIV1824','T_ATIV2529','T_DES1014','T_DES1517','T_DES1824',
                       'T_DES2529','T_ATRASO_0_BASICO','T_ATRASO_0_FUND','T_ATRASO_0_MED','T_FBBAS','T_FBFUND','T_FBMED',
                       'T_FBPRE','T_FBSUPER','T_FLBAS','T_FLFUND','T_FLMED','T_FLPRE','T_FLSUPER','P_CONSTR','P_EXTR','P_MED',
                       'P_SIUP','P_SUPER','P_TRANSF','REN0','REN2','REN3','REN5','T_ATIV','T_DES','T_LIXO','T_DENS',
                       'I_FREQ_PROP','POPT','REN1','P_FUND','T_ATIV18M','T_DES18M','T_SLUZ','PIA18M','POP','E_ANOSESTUDO',
                       'T_FUND18M','PESO18','T_BANAGUA','PAREDE','ESPVIDA','P_AGRO','P_COM','P_SERV','TRABPUB','P_FORMAL',
                       'PEA18M','pesoRUR','pesourb'], axis=1, inplace=True)

In [76]:
# Renomeando as colunas para simplificar
df_idhm3 = df_idhm2.rename(columns={'Codmun6':'municp_cod6',
                                    'Codmun7':'municp_codibge',
                                    'Município':'municp_nome',
                                    
                                    'MORT1':'municp_mort_inf',
                                    'T_AGUA':'municp_domic_agua',
                                    'T_LUZ':'municp_domic_luz',
                                    'AGUA_ESGOTO':'municp_domic_agua&esg',
                                    
                                    'pesotot':'municp_pop_total',
                                    'RENOCUP':'municp_rend_med_ocup',
                                    'RDPC':'municp_rend_pcap',
                                    'T_ANALF18M':'municp_tx_analf',
                                    'I_ESCOLARIDADE':'municp_niv_escol_pop',
                                    
                                    'IDHM' : 'municp_idhm',
                                    'IDHM_E':'municp_idhm_educ',
                                    'IDHM_L':'municp_idhm_longev',
                                    'IDHM_R':'municp_idhm_renda'})
df_idhm3.head()                                       

,municp_cod6,municp_codibge,municp_nome,municp_mort_inf,municp_tx_analf,municp_rend_pcap,municp_rend_med_ocup,municp_domic_agua,municp_domic_luz,municp_domic_agua&esg,municp_pop_total,municp_niv_escol_pop,municp_idhm,municp_idhm_educ,municp_idhm_longev,municp_idhm_renda
0,210005,2100055,AÇAILÂNDIA,23.2,19.22,438.56,974.92,89.57,99.34,17.03,104047,0.464,0.672,0.602,0.785,0.643
1,210010,2100105,AFONSO CUNHA,34.4,38.74,149.78,522.88,67.77,97.20,16.54,5905,0.260,0.529,0.434,0.725,0.471
2,210015,2100154,ÁGUA DOCE DO MARANHÃO,40.6,39.10,172.55,367.65,86.73,97.78,25.56,11581,0.187,0.500,0.363,0.697,0.494
3,210020,2100204,ALCÂNTARA,28.8,24.16,209.27,405.47,73.63,98.34,38.05,21851,0.379,0.573,0.475,0.753,0.525
4,210030,2100303,ALDEIAS ALTAS,35.3,40.35,179.92,350.15,77.26,87.00,30.28,23952,0.231,0.513,0.374,0.720,0.500


In [77]:
list(df_idhm3.columns)

['municp_cod6',
 'municp_codibge',
 'municp_nome',
 'municp_mort_inf',
 'municp_tx_analf',
 'municp_rend_pcap',
 'municp_rend_med_ocup',
 'municp_domic_agua',
 'municp_domic_luz',
 'municp_domic_agua&esg',
 'municp_pop_total',
 'municp_niv_escol_pop',
 'municp_idhm',
 'municp_idhm_educ',
 'municp_idhm_longev',
 'municp_idhm_renda']

In [78]:
df_idhm3.isna().sum()

municp_cod6              0
municp_codibge           0
municp_nome              0
municp_mort_inf          0
municp_tx_analf          0
municp_rend_pcap         0
municp_rend_med_ocup     0
municp_domic_agua        0
municp_domic_luz         0
municp_domic_agua&esg    0
municp_pop_total         0
municp_niv_escol_pop     0
municp_idhm              0
municp_idhm_educ         0
municp_idhm_longev       0
municp_idhm_renda        0
dtype: int64

In [79]:
df_idhm3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   municp_cod6            217 non-null    int64  
 1   municp_codibge         217 non-null    int64  
 2   municp_nome            217 non-null    object 
 3   municp_mort_inf        217 non-null    float64
 4   municp_tx_analf        217 non-null    float64
 5   municp_rend_pcap       217 non-null    float64
 6   municp_rend_med_ocup   217 non-null    float64
 7   municp_domic_agua      217 non-null    float64
 8   municp_domic_luz       217 non-null    float64
 9   municp_domic_agua&esg  217 non-null    float64
 10  municp_pop_total       217 non-null    int64  
 11  municp_niv_escol_pop   217 non-null    float64
 12  municp_idhm            217 non-null    float64
 13  municp_idhm_educ       217 non-null    float64
 14  municp_idhm_longev     217 non-null    float64
 15  municp

## 3.4 Tratamento dataset Estimativa População 2021 (IBGE)

Arquivo no formato xls disponibilizado no site do IBGE com a estimativa da população para 2021. (https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?=&t=downloads).

Download em 11/02/2022.

In [80]:
# Importando planilha da População 
df_pop = pd.read_excel('estimativa_dou_2021.xls', sheet_name=1, header=1, dtype=str)

In [81]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5593 entries, 0 to 5592
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   UF                  5591 non-null   object
 1   COD. UF             5570 non-null   object
 2   COD. MUNIC          5570 non-null   object
 3   NOME DO MUNICÍPIO   5570 non-null   object
 4   POPULAÇÃO ESTIMADA  5570 non-null   object
dtypes: object(5)
memory usage: 218.6+ KB


In [82]:
df_pop.columns = df_pop.columns.str.replace(' ', '') 


In [83]:
df_pop2 = df_pop.copy()
df_pop2['cod_municp'] = df_pop['COD.UF'].astype(str)+df_pop['COD.MUNIC'].astype(str)

In [84]:
df_pop3 = df_pop2.query('UF == "MA"')
df_pop3 = df_pop3.reset_index(drop=True)

In [85]:
df_pop3

,UF,COD.UF,COD.MUNIC,NOMEDOMUNICÍPIO,POPULAÇÃOESTIMADA,cod_municp
0,MA,21,00055,Açailândia,113783,2100055
1,MA,21,00105,Afonso Cunha,6631,2100105
2,MA,21,00154,Água Doce do Maranhão,12731,2100154
3,MA,21,00204,Alcântara,22126,2100204
4,MA,21,00303,Aldeias Altas,26979,2100303
...,...,...,...,...,...,...
212,MA,21,12803,Viana,52852,2112803
213,MA,21,12852,Vila Nova dos Martírios,13800,2112852
214,MA,21,12902,Vitória do Mearim,32956,2112902
215,MA,21,13009,Vitorino Freire,31520,2113009


In [86]:
## Ajustando os nomes dos municípios
df_pop3['nm_municp'] = df_pop3['NOMEDOMUNICÍPIO'].str.normalize('NFKD').str.encode('ascii',
                                                                        errors='ignore').str.decode('utf-8')
df_pop3['nm_municp'] = df_pop3['nm_municp'].apply(lambda x: x.upper())
df_pop3.head()

,UF,COD.UF,COD.MUNIC,NOMEDOMUNICÍPIO,POPULAÇÃOESTIMADA,cod_municp,nm_municp
0,MA,21,00055,Açailândia,113783,2100055,ACAILANDIA
1,MA,21,00105,Afonso Cunha,6631,2100105,AFONSO CUNHA
2,MA,21,00154,Água Doce do Maranhão,12731,2100154,AGUA DOCE DO MARANHAO
3,MA,21,00204,Alcântara,22126,2100204,ALCANTARA
4,MA,21,00303,Aldeias Altas,26979,2100303,ALDEIAS ALTAS


In [87]:
# Renomeando coluna
df_pop3 = df_pop3.rename(columns={'POPULAÇÃOESTIMADA':'pop_estimada'})

In [88]:
# Eliminando as colunas que não serão tratadas neste trabalho
df_pop3.drop(columns=['UF','NOMEDOMUNICÍPIO','COD.UF','COD.MUNIC'],inplace=True)

In [89]:
df_pop3.loc[df_pop3.nm_municp.str.contains('PACO DO LUMIAR')]

,pop_estimada,cod_municp,nm_municp
129,125.265(14),2107506,PACO DO LUMIAR


A planilha original tinha uma observação (nota de rodapé) referente a população estimada da cidade de Paço do Lumiar. \
Removemos a observação.

In [90]:
# Limpando e corrigindo o valor
df_pop3.loc[df_pop3.nm_municp.str.contains('PACO DO LUMIAR'),'pop_estimada'] = '125265'

In [91]:
df_pop3.loc[df_pop3.nm_municp.str.contains('PACO DO LUMIAR')]

,pop_estimada,cod_municp,nm_municp
129,125265,2107506,PACO DO LUMIAR


In [92]:
df_pop3['pop_estimada'] = df_pop3['pop_estimada'].astype(int)
df_pop3['cod_municp'] = df_pop3['cod_municp'].astype(int)

### 3.4.1 Criando atributo cobertura vacinal

In [93]:
# Incorporando o total das vacinações
df_pop3['municp_qtdevac'] = total.values

In [94]:
df_pop3['pop_estimada'] = pd.to_numeric(df_pop3['pop_estimada'], errors='coerce', downcast='integer')

In [95]:
df_pop3['municp_cobert'] = df_pop3['municp_qtdevac'] / df_pop3['pop_estimada']

In [96]:
df_pop3

,pop_estimada,cod_municp,nm_municp,municp_qtdevac,municp_cobert
0,113783,2100055,ACAILANDIA,65960,0.579700
1,6631,2100105,AFONSO CUNHA,4399,0.663399
2,12731,2100154,AGUA DOCE DO MARANHAO,9011,0.707800
3,22126,2100204,ALCANTARA,17856,0.807014
4,26979,2100303,ALDEIAS ALTAS,13841,0.513029
...,...,...,...,...,...
212,52852,2112803,VIANA,32328,0.611670
213,13800,2112852,VILA NOVA DOS MARTIRIOS,6681,0.484130
214,32956,2112902,VITORIA DO MEARIM,19819,0.601378
215,31520,2113009,VITORINO FREIRE,18819,0.597049


In [97]:
df_pop3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pop_estimada    217 non-null    int32  
 1   cod_municp      217 non-null    int32  
 2   nm_municp       217 non-null    object 
 3   municp_qtdevac  217 non-null    int64  
 4   municp_cobert   217 non-null    float64
dtypes: float64(1), int32(2), int64(1), object(1)
memory usage: 6.9+ KB


In [115]:
df_pop3

,pop_estimada,cod_municp,nm_municp,municp_qtdevac,municp_cobert
0,113783,2100055,ACAILANDIA,65960,0.579700
1,6631,2100105,AFONSO CUNHA,4399,0.663399
2,12731,2100154,AGUA DOCE DO MARANHAO,9011,0.707800
3,22126,2100204,ALCANTARA,17856,0.807014
4,26979,2100303,ALDEIAS ALTAS,13841,0.513029
...,...,...,...,...,...
212,52852,2112803,VIANA,32328,0.611670
213,13800,2112852,VILA NOVA DOS MARTIRIOS,6681,0.484130
214,32956,2112902,VITORIA DO MEARIM,19819,0.601378
215,31520,2113009,VITORINO FREIRE,18819,0.597049


## 3.5 Junção dos datasets 

### 3.5.1 Junção datasets IDHM e População dos Municípios

In [98]:
df_pop3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pop_estimada    217 non-null    int32  
 1   cod_municp      217 non-null    int32  
 2   nm_municp       217 non-null    object 
 3   municp_qtdevac  217 non-null    int64  
 4   municp_cobert   217 non-null    float64
dtypes: float64(1), int32(2), int64(1), object(1)
memory usage: 6.9+ KB


In [99]:
df_idhm3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   municp_cod6            217 non-null    int64  
 1   municp_codibge         217 non-null    int64  
 2   municp_nome            217 non-null    object 
 3   municp_mort_inf        217 non-null    float64
 4   municp_tx_analf        217 non-null    float64
 5   municp_rend_pcap       217 non-null    float64
 6   municp_rend_med_ocup   217 non-null    float64
 7   municp_domic_agua      217 non-null    float64
 8   municp_domic_luz       217 non-null    float64
 9   municp_domic_agua&esg  217 non-null    float64
 10  municp_pop_total       217 non-null    int64  
 11  municp_niv_escol_pop   217 non-null    float64
 12  municp_idhm            217 non-null    float64
 13  municp_idhm_educ       217 non-null    float64
 14  municp_idhm_longev     217 non-null    float64
 15  municp

In [100]:
# Juntando os Datasets
df_popidhm = df_pop3.merge(df_idhm3, left_on='cod_municp', right_on='municp_codibge', how='left')

In [101]:
df_popidhm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pop_estimada           217 non-null    int32  
 1   cod_municp             217 non-null    int32  
 2   nm_municp              217 non-null    object 
 3   municp_qtdevac         217 non-null    int64  
 4   municp_cobert          217 non-null    float64
 5   municp_cod6            217 non-null    int64  
 6   municp_codibge         217 non-null    int64  
 7   municp_nome            217 non-null    object 
 8   municp_mort_inf        217 non-null    float64
 9   municp_tx_analf        217 non-null    float64
 10  municp_rend_pcap       217 non-null    float64
 11  municp_rend_med_ocup   217 non-null    float64
 12  municp_domic_agua      217 non-null    float64
 13  municp_domic_luz       217 non-null    float64
 14  municp_domic_agua&esg  217 non-null    float64
 15  municp

In [102]:
df_popidhm.loc[:,['nm_municp','pop_estimada','municp_pop_total']]

,nm_municp,pop_estimada,municp_pop_total
0,ACAILANDIA,113783,104047
1,AFONSO CUNHA,6631,5905
2,AGUA DOCE DO MARANHAO,12731,11581
3,ALCANTARA,22126,21851
4,ALDEIAS ALTAS,26979,23952
...,...,...,...
212,VIANA,52852,49496
213,VILA NOVA DOS MARTIRIOS,13800,11258
214,VITORIA DO MEARIM,32956,31217
215,VITORINO FREIRE,31520,31658


Houve uma evolução na população dos municípios entre as bases do IDHM (AtlasBR) e IBGE.

Vamos considerar os valores do IBGE estimativas 2021.

In [103]:
# Eliminando e renomeando colunas após a junção

df_popidhm.drop(columns=['nm_municp'], axis=1, inplace=True)
df_popidhm.drop(columns=['municp_codibge'], axis=1, inplace=True)
df_popidhm.drop(columns=['municp_pop_total'], axis=1, inplace=True)

df_popidhm = df_popidhm.rename(columns={'pop_estimada':'municp_populacao'})
df_popidhm = df_popidhm.rename(columns={'cod_municp':'municp_codigo'})

In [104]:
df_popidhm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   municp_populacao       217 non-null    int32  
 1   municp_codigo          217 non-null    int32  
 2   municp_qtdevac         217 non-null    int64  
 3   municp_cobert          217 non-null    float64
 4   municp_cod6            217 non-null    int64  
 5   municp_nome            217 non-null    object 
 6   municp_mort_inf        217 non-null    float64
 7   municp_tx_analf        217 non-null    float64
 8   municp_rend_pcap       217 non-null    float64
 9   municp_rend_med_ocup   217 non-null    float64
 10  municp_domic_agua      217 non-null    float64
 11  municp_domic_luz       217 non-null    float64
 12  municp_domic_agua&esg  217 non-null    float64
 13  municp_niv_escol_pop   217 non-null    float64
 14  municp_idhm            217 non-null    float64
 15  municp

In [105]:
df_popidhm.isnull().sum()

municp_populacao         0
municp_codigo            0
municp_qtdevac           0
municp_cobert            0
municp_cod6              0
municp_nome              0
municp_mort_inf          0
municp_tx_analf          0
municp_rend_pcap         0
municp_rend_med_ocup     0
municp_domic_agua        0
municp_domic_luz         0
municp_domic_agua&esg    0
municp_niv_escol_pop     0
municp_idhm              0
municp_idhm_educ         0
municp_idhm_longev       0
municp_idhm_renda        0
dtype: int64

### 3.5.2 Junção datasets PopIDHM e Registros de Vacinação

In [106]:
df_regvac3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822693 entries, 0 to 822692
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   paciente_id          822693 non-null  object        
 1   paciente_idade       822693 non-null  float64       
 2   paciente_dt_nasc     822693 non-null  datetime64[ns]
 3   paciente_sexo        822693 non-null  object        
 4   paciente_raca        822693 non-null  object        
 5   paciente_cod_municp  822693 non-null  object        
 6   paciente_nm_municp   822693 non-null  object        
 7   paciente_categoria   822371 non-null  object        
 8   vacina_nm_municp     822693 non-null  object        
 9   vacina_dt_aplic      822693 non-null  datetime64[ns]
 10  vacina_desc_dose     822693 non-null  object        
 11  vacina_nome          822693 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(9)
memory usage: 75.3+ MB


In [107]:
df_popidhm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   municp_populacao       217 non-null    int32  
 1   municp_codigo          217 non-null    int32  
 2   municp_qtdevac         217 non-null    int64  
 3   municp_cobert          217 non-null    float64
 4   municp_cod6            217 non-null    int64  
 5   municp_nome            217 non-null    object 
 6   municp_mort_inf        217 non-null    float64
 7   municp_tx_analf        217 non-null    float64
 8   municp_rend_pcap       217 non-null    float64
 9   municp_rend_med_ocup   217 non-null    float64
 10  municp_domic_agua      217 non-null    float64
 11  municp_domic_luz       217 non-null    float64
 12  municp_domic_agua&esg  217 non-null    float64
 13  municp_niv_escol_pop   217 non-null    float64
 14  municp_idhm            217 non-null    float64
 15  municp

In [108]:
df_regvac3['paciente_cod_municp'] = df_regvac3['paciente_cod_municp'].astype(int)

In [109]:
# Junção dos Datasets
df_regvac_final = df_regvac3.merge(df_popidhm, left_on='paciente_cod_municp', right_on='municp_cod6', how='inner')

In [110]:
df_regvac_final.head()

,paciente_id,paciente_idade,paciente_dt_nasc,paciente_sexo,paciente_raca,paciente_cod_municp,paciente_nm_municp,paciente_categoria,vacina_nm_municp,vacina_dt_aplic,...,municp_rend_pcap,municp_rend_med_ocup,municp_domic_agua,municp_domic_luz,municp_domic_agua&esg,municp_niv_escol_pop,municp_idhm,municp_idhm_educ,municp_idhm_longev,municp_idhm_renda
0,000012af7d1821b858c18ea6fa5979452a03a075f2b1de...,55.0,1966-09-02,M,SEM INFORMACAO,210330,CODO,Faixa Etária,CODO,2021-11-09,...,273.45,536.85,87.21,97.78,18.2,0.345,0.595,0.492,0.754,0.568
1,000a80f7e537109a5015e304d5fda828226c08ff31c409...,18.0,2003-02-06,F,SEM INFORMACAO,210330,CODO,Faixa Etária,CODO,2021-08-12,...,273.45,536.85,87.21,97.78,18.2,0.345,0.595,0.492,0.754,0.568
2,001ce9ee07bc3bd4d429f95a4e586bf151cb28fae35383...,22.0,1999-09-08,M,PRETA,210330,CODO,Faixa Etária,CODO,2021-09-23,...,273.45,536.85,87.21,97.78,18.2,0.345,0.595,0.492,0.754,0.568
3,0056fa0163c206ccd1a9fdf9c1ccc2a026a556f12f7b23...,30.0,1991-01-09,F,AMARELA,210330,CODO,Faixa Etária,CODO,2021-11-29,...,273.45,536.85,87.21,97.78,18.2,0.345,0.595,0.492,0.754,0.568
4,0078b467fdbb85896fbc89b515d0b4a90dd95690ffaec6...,92.0,1928-07-02,F,AMARELA,210330,CODO,Comorbidades,CODO,2021-04-07,...,273.45,536.85,87.21,97.78,18.2,0.345,0.595,0.492,0.754,0.568


In [111]:
print(df_regvac_final.loc[:,['municp_nome','paciente_cod_municp','municp_cod6','municp_codigo']],'/n')

       municp_nome  paciente_cod_municp  municp_cod6  municp_codigo
0             CODÓ               210330       210330        2103307
1             CODÓ               210330       210330        2103307
2             CODÓ               210330       210330        2103307
3             CODÓ               210330       210330        2103307
4             CODÓ               210330       210330        2103307
...            ...                  ...          ...            ...
736999  BACURITUBA               210135       210135        2101350
737000  BACURITUBA               210135       210135        2101350
737001  BACURITUBA               210135       210135        2101350
737002  BACURITUBA               210135       210135        2101350
737003  BACURITUBA               210135       210135        2101350

[737004 rows x 4 columns] /n


In [112]:
# Eliminando colunas do atributo código do municipio usadas exclusivamente para a junção dos datasets
df_regvac_final.drop(columns=['paciente_cod_municp','municp_cod6'], axis=1, inplace=True)

In [113]:
df_regvac_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737004 entries, 0 to 737003
Data columns (total 28 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   paciente_id            737004 non-null  object        
 1   paciente_idade         737004 non-null  float64       
 2   paciente_dt_nasc       737004 non-null  datetime64[ns]
 3   paciente_sexo          737004 non-null  object        
 4   paciente_raca          737004 non-null  object        
 5   paciente_nm_municp     737004 non-null  object        
 6   paciente_categoria     736696 non-null  object        
 7   vacina_nm_municp       737004 non-null  object        
 8   vacina_dt_aplic        737004 non-null  datetime64[ns]
 9   vacina_desc_dose       737004 non-null  object        
 10  vacina_nome            737004 non-null  object        
 11  municp_populacao       737004 non-null  int32         
 12  municp_codigo          737004 non-null  int3

Dataset final da fase tratamento de dados.

##### Este será nosso dataset de estudo. 

In [114]:
# Exportação do dataset em formato csv
df_regvac_final.to_csv('RegVacinaMA_TratDados.csv')